In [1]:
import spotipy
import spotipy.util as util
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify
from joblib import load
import pickle
import numpy as np

In [2]:
USERNAME = 'augt12ocf9csxa4s8kbq9reg8' #your spotify username
CLIENT_ID = 'b954b92b9a674e6a9cf7322649da82d1' #set at your developer account
CLIENT_SECRET = '920cd41023994b1a99210243ce8967b2' #set at your developer account
REDIRECT_URI = 'https://google.com/' #set at your developer account, usually "http://localhost:8000"
SCOPE = 'user-library-read' # or else
# ps. REDIRECT_URI is crucial here. if http://localhost:8000 is not set, or with a single '/' misplaced, it will generate a connection error.

# then pass them:

token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)

if token:
   sp = spotipy.Spotify(auth=token)

# Flask Predict function

In [37]:
def get_features(song_id,sp):
    results_dict = sp.audio_features(song_id)[0]
    audio_features = {
        "audio_features": {
            key: results_dict[key] for key in results_dict.keys() & {
                'danceability',
                'energy',
                'key',
                'loudness',
                'mode',
                'speechiness',
                'acousticness',
                'instrumentalness',
                'liveness',
                'valence',
                'tempo',
                'time_signature'}}}

    return audio_features

In [58]:
def predict(content):
    similar_songs = []
    print('Loading dataframe...')
    dataframe = pd.DataFrame.from_dict(
        json_normalize(content['audio_features']),
                                orient='columns')
    print(dataframe)
    print('Dataframe Object Created')
    print('Loading pickled scaler...')
    scalar = load('./models/scalar2.joblib')
    print(scalar)
    print('Pickled scaler loaded')
    print('Scaling dataframe object...')
    df_array = np.array(dataframe)
    print(df_array)
    dataframe_scaled = scalar.transform(df_array)
    print(dataframe_scaled)
#     print('Dataframe scaled')
#     print('Loading pickled model...')
#     model = load('./models/model2.joblib')
#     print('Model loaded')
#     results = model.kneighbors([dataframe_scaled][0])[1]
#     print('Prediction executed')
#     print('song_id_list loading...')
#     song_id_list = load('./data/song_id_list2.joblib')
#     print('song_id_list loaded')
    
#     print('beginning for loop...')
#     for song_row in results[0][1:]:
#         song_id = song_id_list[song_row]
#         similar_songs.append({'similarity': [.99], 'values': song_id})
#     json_dict = {"songs": similar_songs}
#     return json_dict

In [50]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from flask import request
from joblib import dump
from joblib import load

import pandas as pd
from pandas.io.json import json_normalize
from flask import jsonify

In [59]:
predict(get_features(song_id,sp))

Loading dataframe...
   energy  loudness  key   tempo  liveness  instrumentalness  mode  \
0   0.652    -4.647    0  75.863     0.142                 0     0   

   danceability  valence  acousticness  speechiness  time_signature  
0         0.253    0.298         0.215       0.0406               4  
Dataframe Object Created
Loading pickled scaler...
MinMaxScaler(copy=True, feature_range=(0, 1))
Pickled scaler loaded
Scaling dataframe object...
[[ 6.5200e-01 -4.6470e+00  0.0000e+00  7.5863e+01  1.4200e-01  0.0000e+00
   0.0000e+00  2.5300e-01  2.9800e-01  2.1500e-01  4.0600e-02  4.0000e+00]]
[[ 6.54618474e-01 -4.66098295e+00  0.00000000e+00  7.58630000e+01
   1.29090909e-02  0.00000000e+00  9.04486252e-01  2.53000000e-01
   3.07851240e-01  8.60058484e-04  8.12000000e-03  4.00000000e+00]]


/Users/user/.local/share/virtualenvs/model_notebooks-inW34qmC/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


# Attemping another approach using dataframe - success

In [306]:
#loop takes KNN results and filters by source track genres
# old
filtered_list = []
# filtered_lists = []
source_song_index = knn_results1[0][0]
source_genre_list = df.loc[source_song_index]["genres_stripped"].split(",")
# print(source_genre_list)
for output_song_index in knn_results1[0][1:]:
#     print(output_song_index)
    output_genre_list = df.loc[output_song_index]["genres_stripped"].split(",")
#     print(output_genre_list)
    for output_genre in output_genre_list:
        output_genre = output_genre.strip(' ')
#         print(output_genre)
        for source_genre in source_genre_list:
            source_genre = source_genre.strip(' ')
#             print(source_genre)
            if source_genre == output_genre:
#                 print(output_song_index,source_genre,output_genre)
                filtered_list.append(output_song_index)
#             print(filtered_list)
            else:
# #             print("source genre does not match output genre!")
                continue
#     filtered_lists.append(filtered_list)
print(len(set(filtered_list)),"!!!!!")

807 !!!!!


In [260]:
len(set(filtered_list))

376

In [308]:
knn_results1[0][0]

2

# Ready for Production - Load in Genre Array and Utilize in updated function for genre filtering

In [ ]:
def filter_model(): 
    #loop takes KNN results and filters by source track genres
    filtered_list = []
    song_list_length = 364
    source_song_index = knn_results1[0][0]
    source_genre_list = genre_array[source_song_index]
    for output_song_index in knn_results1[0][1:]:
        output_genre_list = genre_array[output_song_index]
        for output_genre in output_genre_list:
            for source_genre in source_genre_list:
                if source_genre == output_genre:
                    filtered_list.append(output_song_index)
                else:
                    continue
    if len(set(filtered_list)) > song_list_length:
        print("list large enough")
    else:
        counter = song_list_length - len(set(filtered_list))
        print(len(set(filtered_list)))
        print(counter)
        print(f'need to add {counter} items to final song output')
        for output_song_index in knn_results1[0][1:]:
            if output_song_index not in filtered_list:
                if counter > 0:
                    filtered_list.append(output_song_index)
                    counter -= 1
                else:
                    break
            
#     print(len(output_song_index_list),"!!!!!")    

In [ ]:
filter_model()

# Test Multiple Rows - Functional!

In [272]:
def output_len_of_lists(list_of_lists):
    for list_item in list_of_lists:
        print(len(list_item))

In [ ]:
#loop takes range of KNN results per defined track range and filter matches by source track genres
filtered_lists = []
for source_song_index in range(0,1000):
    knn_results1 = knn1.kneighbors([data_normalized[source_song_index]])[1]
    knn_result_ouputs = knn_results1[0][1:]
    filtered_list = []
    source_genre_list = genre_array[source_song_index]
    for output_song_index in knn_result_ouputs:
        output_genre_list = genre_array[output_song_index]
        for output_genre in output_genre_list:
            for source_genre in source_genre_list:
                if source_genre == output_genre:
                    filtered_list.append(output_song_index)
                else:
                    continue
    filtered_lists.append(set(filtered_list))
    
output_len_of_lists(filtered_lists)

# From predict.py - Production

In [25]:
def instantiate_sp(token):
    sp = spotipy.Spotify(auth=token)
    return sp

In [26]:
instantiate_sp(token)

In [27]:
def get_id(sp):
    results = sp.current_user_saved_tracks()
    song_id = results['items'][0]['track']['id']
    return song_id

In [28]:
song_id = get_id(instantiate_sp(token))

In [30]:
def get_acoustical_features(song_id,sp):
    return sp.audio_features(song_id)[0]

In [31]:
def get_popularity(trackID):
    return sp.track(trackID)['popularity']

In [32]:
def get_artist_id(trackID):
    return sp.track(trackID)['artists'][0]['id']

In [33]:
def get_genres(artistID):
    return sp.artist(artistID)['genres']

# Definining New Class for Use in Predict.py

In [3]:
class Sound_Drip:
    
    def __init__(self, token):
        self.token = token
    
    def instantiate_sp(self,token):
        sp = spotipy.Spotify(auth=token)
        return sp

    def get_id(self, sp):
        results = sp.current_user_saved_tracks()
        song_id = results['items'][0]['track']['id']
        return song_id

    def get_acoustical_features(self,song_id,sp):
        acoustical_features = sp.audio_features(song_id)[0]
        return acoustical_features

    def get_popularity(self, song_id):
        popularity =  sp.track(song_id)['popularity']
        return popularity

    def get_artist_id(self, song_id):
        artist = sp.track(song_id)['artists'][0]['id']
        return artist

    def get_genres(self, artist):
        genre = sp.artist(artist)['genres']
        return genre
    
    def create_feature_object(self,popularity, acoustical_features):
        popularity_dict = {'popularity': popularity}
        song_features = acoustical_features
        song_features.update(popularity_dict)
        song_features = {
    "audio_features": {
        key: song_features[key] for key in song_features.keys() & {
            'popularity',
            'acousticness',
            'danceability',
            'energy',
            'instrumentalness',
            'key',
            'liveness',
            'loudness',
            'mode',
            'speechiness',
            'tempo',
            'time_signature',
            'valence'}}}
        print(song_features)
#         df = pd.DataFrame.from_dict(json_normalize(song_features["audio_features"]),orient='columns')   
#         df = df.reindex(sorted(df.columns), axis=1)
#         feature_array = df.to_numpy()
#         return feature_array
    

In [17]:
#         audio_features = {
#             "audio_features": {
#                 key: raw_results[key] for key in results_dict.keys() & {
#                     'danceability',
#                     'energy',
#                     'key',
#                     'loudness',
#                     'mode',
#                     'speechiness',
#                     'acousticness',
#                     'instrumentalness',
#                     'liveness',
#                     'valence',
#                     'tempo',
#                     'time_signature'}}}

In [4]:
sd = Sound_Drip(token)

In [5]:
sp = sd.instantiate_sp(token)

In [6]:
song_id = sd.get_id(sp)

In [7]:
acoustical_features = sd.get_acoustical_features(song_id, sp)

In [8]:
popularity = sd.get_popularity(song_id)

In [9]:
artist = sd.get_artist_id(song_id)

In [10]:
genre = sd.get_genres(artist)

In [16]:
song_features = sd.create_feature_object(popularity, acoustical_features)

{'audio_features': {'mode': 0, 'liveness': 0.142, 'loudness': -4.647, 'time_signature': 4, 'danceability': 0.253, 'instrumentalness': 0, 'popularity': 39, 'key': 0, 'acousticness': 0.215, 'valence': 0.298, 'tempo': 75.863, 'energy': 0.652, 'speechiness': 0.0406}}


In [18]:
print(song_features)

None


In [13]:
## Updated function - Not in Flask - 

def predictfunc(song_features):
    dataframe = pd.DataFrame.from_dict(
        json_normalize(song_features['audio_features']),
                                orient='columns')
    scaler = MinMaxScaler()
    print('Scaling data...')
    data_scaled = scaler.fit_transform(song_features)
    print(data_scaled)
    print('Data scaled')
    normalizer = Normalizer()
    print('Normalizing data...')
#     data_normalized = normalizer.fit_transform(data_scaled)
#     print('Data normalized')
#     print('Loading pickled model...')
#     model = load('models/model2.joblib')
#     print('Model loaded')
# #     need to add popularity and genre filtering here
    
#     results = model.kneighbors([data_scaled][0])[1]
# #     genre filtering

#     print('Prediction executed')
#     print('song_id_list loading...')
#     song_id_list = pickle.load(open('data/old_song_array.pkl', 'rb'))
#     print('song_id_list loaded')

#     print('beginning for loop...')
#     for song_row in results[0][1:]:
#         song_id = song_id_list[song_row]
#         similar_songs.append({'similarity': [.99], 'values': song_id})
#     json_dict = {"songs": similar_songs}
#     return json_dict

In [14]:
predictfunc(song_features)

TypeError: 'NoneType' object is not subscriptable

# For prediction.py - New Functions

In [ ]:
def filter_model(): 
    #loop takes KNN results and filters by source track genres
    filtered_list = []
    song_list_length = 20
    source_genre_list = genre_array[source_song_index]
    for output_song_index in knn_results1[0][1:]:
        output_genre_list = genre_array[output_song_index]
        for output_genre in output_genre_list:
            for source_genre in source_genre_list:
                if source_genre == output_genre:
                    filtered_list.append(output_song_index)
                else:
                    continue
    if len(set(filtered_list)) > song_list_length:
        print("list large enough")
    else:
        counter = song_list_length - len(set(filtered_list))
        print(len(set(filtered_list)))
        print(counter)
        print(f'need to add {counter} items to final song output')
        for output_song_index in knn_results1[0][1:]:
            if output_song_index not in filtered_list:
                if counter > 0:
                    filtered_list.append(output_song_index)
                    counter -= 1
                else:
                    break
            
#     print(len(output_song_index_list),"!!!!!")    

In [53]:
# import numpy as np

# old_song_list = load('data/song_id_list2(1).joblib')

# old_song_array = np.array(old_song_list)

# pickle.dump(old_song_array, open('data/old_song_array.pkl', 'wb'))